In [28]:
# store each doc in a seperated file
doc_id_dict = {}
raw_document = []

with open('./NFCorpus/test.docs', 'r') as f:
    lines = f.readlines()
    index = 0
    for line in lines:
        id, doc = line.split('\t')
        
        doc_id_dict[id] = index
        index += 1
        raw_document.append(doc)
        
    f.close()

In [29]:
import nltk
import os
from nltk.stem import PorterStemmer
import string
from nltk.corpus import stopwords
import numpy as np


term_freq = {}# số lần xuất hiện của các term cả tài liệu Cranfield
doc_freq = {}# số tài liệu xuất hiện của các term mà ta xét
ps = PorterStemmer()
noise = string.punctuation + '0123456789'


nltk.download('stopwords')
stop_words = stopwords.words('english')

def preProcessing(content):
  # loại bỏ ký hiệu và số tách thành mãng
  words = content.translate(str.maketrans("", "", noise)).split()
  # stemming
  words = [ps.stem(i) for i in words]
  # loại bỏ stopwords
  words = [word for word in words if word not in stop_words]
  return words




[nltk_data] Downloading package stopwords to /Users/macos/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [30]:
document = []
for doc in raw_document:
    document.append(preProcessing(doc))


In [31]:
import re
# store each doc in a seperated file
raw_query = {}

with open('./NFCorpus/test.all.queries', 'r') as f:
    lines = f.readlines()    
    for line in lines:
        id, query = line.split('\t')        
                
        raw_query[id] = query
        
    f.close()

# Sort the queries based on the numeric part of the IDs in ascending order
sorted_queries = [content for _, content in sorted(raw_query.items(), key=lambda x: int(re.findall(r'\d+', x[0])[0]))]

queries = []
for query in sorted_queries:        
    preProcessed_query = preProcessing(query)    
    queries.append(preProcessed_query)

    

In [32]:
import math

# Code của bạn
index = {}  # Chỉ mục

# Xây dựng termtable
termtable = []
for i in range(len(document)):
    for term in document[i]:
        termtable.append((term, i+1))

termtable = sorted(termtable, key=lambda x: x[0])

# Tính toán TF
termfreqtable = []
row = termtable[0]
freq = 1
for i in range(1, len(termtable)):
    if row == termtable[i]:
        freq += 1
    else:
        termfreqtable.append((row[0], row[1], freq))
        row = termtable[i]
        freq = 1
termfreqtable.append((row[0], row[1], freq))


# Tính toán IDF
doc_freq = {}  # Số tài liệu chứa thuật ngữ
total_docs = len(document)  #

for term, _, _ in termfreqtable:
    if term not in doc_freq:
        doc_freq[term] = 1
    else:
        doc_freq[term] += 1

idf = {}  # IDF cho mỗi thuật ngữ

for term in doc_freq:
    idf[term] = math.log((total_docs+1) / (doc_freq[term]+1))+1


document_term = {}
for term, doc_id, freq in termfreqtable:
    tfidf = freq * idf[term]

    if term not in index:
        index[term] = {}
    if doc_id not in document_term:
      document_term[doc_id] = {}
    index[term][doc_id] = tfidf
    document_term[doc_id][term] = tfidf

for doc in document_term:
    values = list(document_term[doc].values())
    norm2 = np.linalg.norm(values, ord=2)
    document_term[doc] = {key: value / norm2 for key, value in document_term[doc].items()}#normalize by norm

In [33]:
for term in index:
      for doc in index[term]:
         index[term][doc] = document_term[doc][term]

In [34]:
index

{'aa': {160: 0.44672569481545227,
  314: 0.061114116196176994,
  384: 0.2883006159590034,
  694: 0.23382889785393973,
  1158: 0.3933515817435821,
  1844: 0.2346071262342847,
  2641: 0.3116726015112785,
  2645: 0.12541908093207604,
  2648: 0.18755435861108097,
  2737: 0.45698969739381007,
  2982: 0.1040898562483889,
  3155: 0.18755435861108097},
 'aaa': {2595: 0.6213562731826873},
 'aacr': {83: 0.08302702204857741,
  1963: 0.11149437762723488,
  1992: 0.12546126171126878,
  1994: 0.06227385602615848,
  2512: 0.08811653483923823,
  2664: 0.07256078086531947},
 'aad': {1124: 0.13767563570917998,
  1954: 0.31887939582516484,
  1961: 0.19412592698368059},
 'aaepa': {2641: 0.06691720474386435},
 'aahb': {1161: 0.264174722698629},
 'aaph': {743: 0.08515253506218409, 2702: 0.1282106347983162},
 'aaphinduc': {2711: 0.052244677154096926},
 'aarelev': {2737: 0.08410054011271254},
 'aarp': {2802: 0.1594472371357497,
  2804: 0.07273122165748847,
  3111: 0.06616293614618371},
 'ab': {578: 0.07876166

In [35]:
import math

# hàm này dùng để chuyển câu truy vấn dạng vector
def calculate_query_vector(query, index_list, idf):
    # Tính toán TF của câu truy vấn
    query_freq = {}
    vector_q = {}
    query_length = len(query)
    for term in query:
        if term not in query_freq:
            query_freq[term] = 1
        else:
            query_freq[term] += 1


    for term in query_freq:
      if term in index_list:
        vector_q[term] = query_freq[term]*idf[term]
    squared_sum = sum(value ** 2 for value in vector_q.values())

# Lấy căn bậc hai của tổng bình phương5
    norm = math.sqrt(squared_sum)

# Chia tất cả các giá trị trong từ điển cho norm
    normalized_q = {key: value / norm for key, value in vector_q.items()}
    return normalized_q

# tính độ tương đồng giữa 2 vector
def cosine_similarity(vector1, vector2):
    dot_product = 0.0
    norm1 = 0.0
    norm2 = 0.0

    for key in vector1:
        if key in vector2:
            dot_product += vector1[key] * vector2[key]
        norm1 += vector1[key] ** 2

    for key in vector2:
        norm2 += vector2[key] ** 2
    similarity = dot_product / (math.sqrt(norm1) * math.sqrt(norm2))
    return similarity

# hàm tình dùng để tính độ tương đồng giữa câu truy vấn với các tài liệu
def query_document_similarity(query, index, document_term, threshold, idf):
    # Biểu diễn câu truy vấn dưới dạng vector (sử dụng TF-IDF)
    query_vector = calculate_query_vector(query, index, idf)

    relate_doc = []
    for term in query:
        if term in index:
          for doc_id in index[term]:
            if doc_id not in relate_doc:
              relate_doc.append(doc_id)

    # Tính độ tương đồng giữa câu truy vấn và các tài liệu
    similarities = {}
    for doc_id in relate_doc:
      similarity = cosine_similarity(query_vector, document_term[doc_id])
      similarities[doc_id] = similarity
    sorted_similarities = sorted(similarities.items(), key=lambda item: item[1], reverse=True)
    result = [key for key, value in sorted_similarities if value > threshold]
    return result

In [36]:
predict_results = []
threshold = 0.2
for q in queries:
    predict_results.append(query_document_similarity(q, index, document_term, threshold, idf))

In [37]:
print(predict_results)

[[1, 1164, 2, 1166, 739, 2597, 788, 2515, 744, 206, 1171, 2809, 1160, 452, 1850, 156, 2076, 462, 2657, 1230, 2234, 1154, 1162, 1032, 2596, 2099, 1163, 789, 731, 1167, 2272, 905, 2397, 738, 2097, 1574, 1962, 2664, 3143, 1173, 2512, 2811, 2075, 2078, 2696, 2084, 2440, 168, 2223, 1199, 1159, 3045, 688, 2310, 1969], [1213], [1283, 1289, 1461, 1526, 76, 1282, 3, 1290, 2900, 1874, 2890], [1341, 169, 1065], [1398, 2947, 1389, 1425, 1417, 1274, 1402, 1395, 1410, 1386, 1392, 1415, 1401, 766, 1420, 1426, 1413, 1017, 1418], [2468, 1456, 1457, 900, 1453, 2451, 2425], [1506, 1358, 1091, 671, 2756, 678, 2781, 1203], [6, 1554, 1550, 1555, 3151, 2422, 1551, 1547, 2415, 1503, 1552, 1631, 1558, 1636, 1638, 1236], [1607, 2799, 1096, 2732, 2918], [1658, 2957, 912, 295, 739, 327, 1235, 1659, 210, 2411, 154], [1718, 1279, 1719, 1509, 37], [1792, 2474, 1787, 886, 1786, 1782, 1486, 1202, 1793, 1783, 1781], [1850, 1969, 1849], [1892, 1895, 1893], [1946, 1958, 1944, 1950, 1961, 1947, 1956, 1959, 1954, 1957, 274

In [38]:
result_file = "./NFCorpus/test.2-1-0.qrel"  # Replace with the actual path to your result file

query_doc_ids = {}
with open(result_file, "r") as file:
    for line in file:
        line = line.strip().split("\t")
        query_id = line[0]
        doc_id = line[2]
        if query_id in query_doc_ids:
            query_doc_ids[query_id].append(doc_id)
        else:
            query_doc_ids[query_id] = [doc_id]

In [39]:
real_results = []
for res in query_doc_ids:
    converted_doc_id = []    
    for doc_id in query_doc_ids[res]:
        converted_doc_id.append(doc_id_dict[doc_id])
    
    real_results.append(converted_doc_id)

In [40]:
# Kiểm tra tài liều đó có nằm trong tài liệu thực không truy vẫn thực không ---> danh sách true false của tài liệu truy vấn của mình(OK)
# tính từng cặp r và p ----> bảng r p
# tính AP của từng câu truy vấn
# cuối cùng tính MAP
# lấy dữ liệu RES để tính MAP nhưng chưa xong--------


#Hàm lấy index true
def index_true(lis):
  return [i for i in range(len(lis)) if lis[i] == True]

# Hàm này dùng để tính R và P của các câu truy vấn
def RP(real_results, predict_results):

#Tính R và P của từng câu truy vấn
  check_results = [[predict_results[i][j] in real_results[i] for j in range(len(predict_results[i]))] for i in range(len(predict_results))]# kiểm tra xem câu tìm được có đúng kết quả không
  index_true_results = [index_true(i) for i in check_results]#đây là index của các dự đoán đúng dùng để tính độ chính xác Precision
  len_results = [len(i) for i in real_results]#số kết quả "thực" tìm được của mỗi câu truy vấn để tính độ phủ Recall
  R_P_results = [[((j+1)/len_results[i], (j+1)/(index_true_results[i][j]+1)) for j in range(len(index_true_results[i]))] for i in range(len(index_true_results))]#kết quả R và P của mỗi câu truy vấn
  return R_P_results

def Recall(real_results, predict_results):
  check_results = [[predict_results[i][j] in real_results[i] for j in range(len(predict_results[i]))] for i in range(len(predict_results))]# kiểm tra xem câu tìm được có đúng kết quả không
  len_results = [len(i) for i in real_results]#số kết quả "thực" tìm được của mỗi câu truy vấn
  recall = [(check_results[i].count(True))/len_results[i] for i in range(len(check_results)) if check_results[i]]
  return sum(recall)/len(real_results)

def Precision(real_results, predict_results):
  check_results = [[predict_results[i][j] in real_results[i] for j in range(len(predict_results[i]))] for i in range(len(predict_results))]# kiểm tra xem câu tìm được có đúng kết quả không
  len_predict = [len(i) for i in predict_results]#số kết quả "dự đoán" tìm được của mỗi câu truy vấn
  precision = [(check_results[i].count(True))/len_predict[i] for i in range(len(check_results)) if check_results[i]]
  return sum(precision)/len(real_results)

R_P_results = RP(real_results, predict_results)

# đây là 11 điểm nội suy của TREC
R = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]

#Tính AP của từng câu truy vấn từ R_P_results
# P nội suy là Max(Pr') trong đó r'>=r(đang xét)

#hàm check_r dùng để kiểm tra và lấy ra index của r trong rp
# Lấy index của R đang xét để tìm giá trị P max từ index đó trở về sau
def check_r(r, rp):
  for i in range(len(rp)):
    if rp[i][0]>=r:
      return i
  return -1
# Lấy P max
def max_P(check, rp):
  maxx = 0
  for i in rp[check:]:
    if i[1]>maxx:
      maxx = i[1]
  return maxx


def MAP_11(R_P_results, R):

  AP_results = []
  for i in range(len(R_P_results)):
    ap = 0
    for j in range(11):
      Check = check_r(R[j],R_P_results[i])
      if Check != -1:
        ap = ap + max_P(Check,R_P_results[i])
    AP_results.append(ap/11)

  MAP = sum(AP_results)/225
  return MAP

MAP_11(R_P_results, R), Recall(real_results, predict_results), Precision(real_results, predict_results)


(0.116007581870335, 0.10171984897429813, 0.22475829075958115)